# [OpenAI Cookbook 예제](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb) 참조

In [71]:
import pandas as pd
import tiktoken
import openai
from openai import OpenAI
from scipy.spatial.distance import cosine
import os
client = OpenAI()
from pprint import pprint
import json
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
# from dotenv import load_dotenv
# load_dotenv()
# openai_api_key = os.getenv("OPENAI_API_KEY")
# if openai_api_key is True:
#     openai_api_key = input("OpenAI API Key 입력: ")
#     os.environ["OPENAI_API_KEY"] = openai_api_key
# openai.api_key = os.environ["OPENAI_API_KEY"]

### 레시피 데이터 불러오기

In [52]:
contents_df = pd.read_csv("contents_df.csv")
contents_df.head()

,title,content,expression
0,개발 팀장,"21310## 한국해양기상기술## 한국 해양 기상 기술은 해양,기상환경의 분석, 예...","Spring Boot, Thymeleaf, Java, Spring, Docker, ..."
1,[Moloco] Senior Data Scientist,21110## 몰로코## 몰로코는 모바일 광고 기반의 머신러닝 솔루션 기업으로서 지...,"R, MachineLearning, Python, SQL, Pandas, 빅데이터 ..."
2,AI 엔지니어,"20853## 피매치## ""고객이 자기주도적으로 웰니스를 실현하는 세상을 꿈꾸는 피...","Linux, TensorFlow, Python, Docker, PyTorch, 인공..."
3,JAVA 백엔드 개발자(차장급),21851## 아파트아이## 아파트아이란? 아파트아이는 아파트 단지를 기반으로 고...,"JavaScript, Spring Framework, RDB, Kafka, Java..."
4,[5년 이상 리더급] 풀스택 소프트웨어 엔지니어,22041## 다이얼로그디자인에이전시## dda Inc.는 2016년 창업한 임직원...,"GCP, Git, Django, React, AWS, Flask, Python, A..."


### Few Shot Learning용 예제 만들기

In [53]:
training_data = []

system_message = "You are a helpful recipe assistant. You are to answer the necessary IT recruitment related information from the given information."

def create_user_message(row):
    return f"""Title: {row['title']}\n\information: {row['content']} """

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    # assistant의 content가 적절하지 않음,,
    messages.append({"role": "assistant", "content": row["expression"]})

    return {"messages": messages}

pprint(prepare_example_conversation(contents_df.iloc[0]))

{'messages': [{'content': 'You are a helpful recipe assistant. You are to '
                          'extract the necessary IT recruitment related '
                          'information from the given information.',
               'role': 'system'},
              {'content': 'Title: 개발 팀장\n'
                          '\\information: 21310## 한국해양기상기술## 한국 해양 기상 기술은 '
                          '해양,기상환경의 분석, 예측에 대한 과학적 자료 제공 관련 기술을 바탕으로 응용소프트웨어, '
                          'DB 개발 구축과 하드웨어 개발을 통한 통합솔루션을 구축하는 업체 입니다. 기상청, 항공 '
                          '기상청, 해양 경찰청 등 정부 기관 관련 다양한 GIS 프로젝트 및 R&D를 주로 하고 '
                          '있습니다. 고객사 파견이나 상주 근무를 하지 않고, 사내 개발을 하고 있습니다. 프로젝트 '
                          '개발 업무를 리딩할 팀장급 경력자 분을 찾고 있습니다. 많은 지원 부탁 드립니다.##  '
                          '개발자들에 대한 technical leading 및 지원  기상, 해양 분야 프로젝트 및 '
                          'R&D 개발  개발 팀 문화 및 프로세스 Development##  7년 이상 백엔드 개발 '
                          '및 서비스 운영 경험이 있는 분 혹은 그에 준하신 분  Java 기술 '
       

<>:6: SyntaxWarning: invalid escape sequence '\i'
<>:6: SyntaxWarning: invalid escape sequence '\i'
C:\Users\r2com\AppData\Local\Temp\ipykernel_14820\3828387410.py:6: SyntaxWarning: invalid escape sequence '\i'
  return f"""Title: {row['title']}\n\information: {row['content']} """


### 성능 측정 목적으로 Training Data와 Test Data 를 나누기

In [60]:
training_df = contents_df.loc[0:100]

# apply the prepare_example_conversation function to each row of the training_df
training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the necessary IT recruitment related information from the given information.'}, {'role': 'user', 'content': "Title: 개발 팀장\n\\information: 21310## 한국해양기상기술## 한국 해양 기상 기술은 해양,기상환경의 분석, 예측에 대한 과학적 자료 제공 관련 기술을 바탕으로 응용소프트웨어, DB 개발 구축과 하드웨어 개발을 통한 통합솔루션을 구축하는 업체 입니다. 기상청, 항공 기상청, 해양 경찰청 등 정부 기관 관련 다양한 GIS 프로젝트 및 R&D를 주로 하고 있습니다. 고객사 파견이나 상주 근무를 하지 않고, 사내 개발을 하고 있습니다. 프로젝트 개발 업무를 리딩할 팀장급 경력자 분을 찾고 있습니다. 많은 지원 부탁 드립니다.##  개발자들에 대한 technical leading 및 지원  기상, 해양 분야 프로젝트 및 R&D 개발  개발 팀 문화 및 프로세스 Development##  7년 이상 백엔드 개발 및 서비스 운영 경험이 있는 분 혹은 그에 준하신 분  Java 기술 생태계(Java/Spring/JPA)에 대한 이해도와 엔지니어링 역량(OOP/TDD/Refactoring)에 관심이 있으신 분  비즈니스 마인드 및 커뮤니케이션 역량을 소유한 분  소규모 팀을 리드 하면서 팀원을 코칭하고 양성한 경험이 있는 분##  개발팀장 혹은 개발 리더 경험이 있으신 분  레거시 시스템 개선 경험 및 대용량 시스템 운영 경험이 있는 분  2D/3D 공간 정보에 대한 이해가 있으신 분  Geoserver, QGIS, Openlayers, Cesium, GDAL 개발 경험이 있으신 분  DDD, Agile, Docker, Kubernetes, MSA, Kafka, Elastic Sear

In [61]:
validation_df = contents_df.loc[101:200]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

### 데이터들 임시 저장!

In [62]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [63]:
training_file_name = "tmp_job_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_job_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [15]:
!head -n 5 tmp_recipe_finetune_training.jsonl


'head'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


### Training 파일들과 Validation 파일들을 둘 다 파일로 집어 넣음

In [72]:
with open(training_file_name, "rb") as training_fd:
    training_response = openai.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(validation_file_name, "rb") as validation_fd:
    validation_response = openai.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-WRRXqGbeKbZi0OldOUi5FEA4
Validation file ID: file-UaF8OR8g0hjvn7bZFyFLuAtQ


In [73]:
response = openai.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="job-position",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)


Job ID: ftjob-7e4fAnrbFsb4IA0tISWhedjU
Status: validating_files


Job ID: ftjob-wFspY1N9VVybdBN59xj2t88c

In [84]:
response = openai.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-7e4fAnrbFsb4IA0tISWhedjU
Validating training file: file-WRRXqGbeKbZi0OldOUi5FEA4 and validation file: file-UaF8OR8g0hjvn7bZFyFLuAtQ
File file-WRRXqGbeKbZi0OldOUi5FEA4 contains examples greater than the supported context size for model `gpt-3.5-turbo-0613` (4096 tokens)
Files validated, moving job to queued state
Fine-tuning job started
The job failed due to an internal error, re-enqueued for retry


In [78]:
response = openai.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

RuntimeError: Fine-tuned model ID not found. Your job has likely not been completed yet.

In [58]:
test_df = contents_df.loc[201:300]
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": create_user_message(test_row)})

pprint(test_messages)

[{'content': 'You are a helpful recipe assistant. You are to extract the '
             'necessary IT recruitment related information from the given '
             'information.',
  'role': 'system'},
 {'content': 'Title: Cloud Back-End 개발자\n'
             '\\information: 22291## 이우소프트## 팀 소개  이우소프트의 개발실은 치과 진단용 장비를 통해 '
             '얻은 이미지 정보를 처리하는 다양한 소프트웨어를 개발하는 이우소프트의 핵심 부서로, 17명이 근무하고 있습니다.  '
             '개발실은 이우소프트의 Desktop/Web,Cloud 기반 진단 SW 뿐 아니라, 진단 데이터를 관리하는 서버군 '
             '제품, 그리고 치과 업무가 원활하게 진행될 수 있도록 기공소 연계 SW, 소모품 주문 SW 등 다양한 소프트웨어를 '
             '개발하고 있습니다.  이번에 채용하는 Desktop GUI App 개발자는 이우소프트에서 개발 중인 차세대 진단 '
             '소프트웨어를 중심으로 기타 다양한 Desktop 기반 GUI App들을 개발하시게 됩니다.  ## 개발실장 '
             'Elly의 한 마디  "이우소프트의 개발실은 항상 새로운 제품과 영역에 도전합니다. 지금 이 순간도 Desktop '
             '기반 SW에서 Web/Cloud 기반 SW 영역으로 도전하는 과정이죠. 이런 과정에서 재미있게 새로운 영역과 기술에 '
             '관심을 가지고, 학습하고, 의견을 내며 주도적으로 일하시는 분이셨으면 좋겠어요.   또한, 저희 Product는 '
             '민감한 의료 정보를 다루는 SW인만큼, 사전 설계와 

In [75]:
response = openai.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `ft:gpt-3.5-turbo-0613:sesac:job-position:8lCAJbqk` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}